Following the tutorial for CAIMAN 3D segmentation

In [ ]:
import pickle
from caiman import load_memmap
import numpy as np
import matplotlib.pyplot as plt
from IPython import get_ipython
import logging
from scipy.ndimage import gaussian_filter
from tifffile.tifffile import imwrite

import caiman as cm
from caiman.utils.visualization import nb_view_patches3d
import caiman.source_extraction.cnmf as cnmf
from caiman.paths import caiman_datadir

try:
    if __IPYTHON__:
        get_ipython().run_line_magic('load_ext', 'autoreload')
        get_ipython().run_line_magic('autoreload', '2')
except NameError:
    pass

import bokeh.plotting as bpl
bpl.output_notebook()

logging.basicConfig(format=
                    "%(relativeCreated)12d [%(filename)s%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    level=logging.WARNING)

In [ ]:
# load in data (remember to subtract the minimum value so that the scan is positive (necessary for CNMF)

filename = 'onefield_miniscan.npy'
scan = np.load(filename)
scan -= np.min(scan)
print(scan.shape, np.min(scan))

In [ ]:
# get scan into the shape expected by the segmentation method

Y = np.swapaxes(scan, 0, 3)
Y2 = np.moveaxis(scan, 2, 0)
dims = Y.shape[1:]
print(Y2.shape)
Cn = cm.local_correlations(Y, swap_dim=False)

In [ ]:
d1, d2, d3 = dims
x, y = (int(1.2 * (d1 + d3)), int(1.2 * (d2 + d3)))
scale = 6/x
fig = plt.figure(figsize=(scale*x, scale*y))
axz = fig.add_axes([1-d1/x, 1-d2/y, d1/x, d2/y])
plt.imshow(Cn.max(2).T, cmap='gray')
plt.title('Max.proj. z')
plt.xlabel('x')
plt.ylabel('y')
axy = fig.add_axes([0, 1-d2/y, d3/x, d2/y])
plt.imshow(Cn.max(0), cmap='gray')
plt.title('Max.proj. x')
plt.xlabel('z')
plt.ylabel('y')
axx = fig.add_axes([1-d1/x, 0, d1/x, d3/y])
plt.imshow(Cn.max(1).T, cmap='gray')
plt.title('Max.proj. y')
plt.xlabel('x')
plt.ylabel('z');
plt.show()

In [ ]:
#%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='multiprocessing', n_processes=8, single_thread=False)

In [ ]:
# save to a tiff file, which the tutorial wants you to do

fname = 'caiman-tutorial/scanMovieOneFieldAllFrames.tif'
imwrite(fname, Y2)

In [ ]:
# motion correction parameters
opts_dict = {'fnames': fname,
             'strides': (24, 24, 6),    # start a new patch for pw-rigid motion correction every x pixels
            'overlaps': (12, 12, 2),   # overlap between patches (size of patch strides+overlaps)
            'max_shifts': (4, 4, 2),   # maximum allowed rigid shifts (in pixels)
            'max_deviation_rigid': 5,  # maximum shifts deviation allowed for patch with respect to rigid shifts
            'pw_rigid': False,         # flag for performing non-rigid motion correction
            'is3D': True}

opts = cnmf.params.CNMFParams(params_dict=opts_dict)

In [ ]:
# first we create a motion correction object with the parameters specified
mc = cm.motion_correction.MotionCorrect(fname, dview=dview, **opts.get_group('motion'))
# note that the file is not loaded in memory

In [ ]:
%%capture
#%% Run motion correction using NoRMCorre
mc.motion_correct(save_movie=True)

In [ ]:
plt.figure(figsize=(12,3))
plt.subplot(1,1,1)
for k in (0,1,2):
    plt.plot(np.array(mc.shifts_rig)[:,k], label=('x','y','z')[k])
plt.legend()
plt.title('inferred shifts')
plt.xlabel('frames')
plt.ylabel('pixels')

In [ ]:
#%% MEMORY MAPPING
# memory map the file in order 'C'
fname_new = cm.save_memmap(mc.mmap_file, base_name='memmap_', order='C',
                           border_to_0=0, dview=dview) # exclude borders

# now load the file
Yr, dims, T = cm.load_memmap(fname_new)
images = np.reshape(Yr.T, [T] + list(dims), order='F') 
    #load frames in python format (T x X x Y)

In [ ]:
#%% restart cluster to clean up memory
cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='multiprocessing', n_processes=8, single_thread=False)

In [ ]:
# set parameters
patch_size = np.array([20,20,10])
num_background_components = 1
merge_threshold = 0.7
fps = 8.3091
proportion_patch_overlap = 0.2
num_components_per_patch = 8
init_method = 'greedy_roi'
soma_diameter = np.array([3.2,3.2,3])
num_pixels_per_process = 1000

half_patch_size = np.int32(np.round(patch_size/2))
patch_overlap = np.int32(np.round(patch_size*proportion_patch_overlap))
patch_overlap[-1] = 2

rf = half_patch_size  # half-size of the patches in pixels. rf=25, patches are 50x50
stride = patch_overlap  # amount of overlap between the patches in pixels
K = num_components_per_patch  # number of neurons expected per patch
gSig = soma_diameter/2  # expected half size of neurons
merge_thresh = merge_threshold  # merging threshold, max correlation allowed
p = 2  # order of the autoregressive system
print('set')

In [ ]:
%%time
cnm = cnmf.CNMF(n_processes, 
                k=K, 
                gSig=gSig, 
                merge_thresh=merge_thresh, 
                p=p, 
                dview=dview,
                rf=rf, 
                stride=stride, 
                only_init_patch=True)
cnm.params.set('spatial', {'se': np.ones((3,3,1), dtype=np.uint8)})
cnm = cnm.fit(images)
print(('Number of components:' + str(cnm.estimates.A.shape[-1])))

In [ ]:
#%% COMPONENT EVALUATION
# the components are evaluated in two ways:
#   a) the shape of each component must be correlated with the data
#   b) a minimum peak SNR is required over the length of a transient

fr = 10 # approx final rate  (after eventual downsampling )
decay_time = 1.  # length of typical transient in seconds 
use_cnn = False  # CNN classifier is designed for 2d (real) data
min_SNR = 3      # accept components with that peak-SNR or higher
rval_thr = 0.6   # accept components with space correlation threshold or higher
cnm.params.change_params(params_dict={'fr': fr,
                                      'decay_time': decay_time,
                                      'min_SNR': min_SNR,
                                      'rval_thr': rval_thr,
                                      'use_cnn': use_cnn})

cnm.estimates.evaluate_components(images, cnm.params, dview=dview)

In [ ]:
print(('Keeping ' + str(len(cnm.estimates.idx_components)) +
       ' and discarding  ' + str(len(cnm.estimates.idx_components_bad))))

In [ ]:
%%time
cnm.params.set('temporal', {'p': p})
cnm2 = cnm.refit(images)

In [ ]:
cnm2.estimates.nb_view_components_3d(image_type='corr', 
                                     dims=dims, 
                                     Yr=Yr, 
                                     denoised_color='red', 
                                     max_projection=True,
                                     axis=2);

In [ ]:
cnm2.estimates.nb_view_components_3d(image_type='mean', dims=dims, axis=2);


In [ ]:
cnm2.estimates.A[:,59].toarray().reshape((240,240,10)).sum(axis=(0,1))

In [ ]:
#%% MEMORY MAPPING
# memory map the file in order 'C'
fname_2d = r'data/caiman/caiman_d1_240_d2_240_d3_1_order_C_frames_2500_.mmap'

# now load the file
Yr2d, dims2d, T2d = cm.load_memmap(fname_2d)
images = np.reshape(Yr2d.T, [T2d] + list(dims2d), order='F') 
images -= np.min(images)
    #load frames in python format (T x X x Y)

In [ ]:
#%% restart cluster to clean up memory
cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='multiprocessing', n_processes=8, single_thread=False)

In [ ]:
# set parameters
patch_size = np.array([20,20])
num_background_components = 1
merge_threshold = 0.7
fps = 8.3091
proportion_patch_overlap = 0.2
num_components_per_patch = 8
init_method = 'greedy_roi'
soma_diameter = np.array([3.2,3.2])
num_pixels_per_process = 1000

half_patch_size = np.int32(np.round(patch_size/2))
patch_overlap = np.int32(np.round(patch_size*proportion_patch_overlap))
patch_overlap[-1] = 2

rf = half_patch_size  # half-size of the patches in pixels. rf=25, patches are 50x50
stride = patch_overlap  # amount of overlap between the patches in pixels
K = num_components_per_patch  # number of neurons expected per patch
gSig = soma_diameter/2  # expected half size of neurons
merge_thresh = merge_threshold  # merging threshold, max correlation allowed
p = 2  # order of the autoregressive system
print('set')

In [ ]:
%%time
cnm2d = cnmf.CNMF(n_processes, 
                k=K, 
                gSig=gSig, 
                merge_thresh=merge_thresh, 
                p=p, 
                dview=dview,
                rf=rf, 
                stride=stride, 
                only_init_patch=True)
cnm2d = cnm2d.fit(images)
print(('Number of components:' + str(cnm2d.estimates.A.shape[-1])))

In [ ]:
#%% COMPONENT EVALUATION
# the components are evaluated in two ways:
#   a) the shape of each component must be correlated with the data
#   b) a minimum peak SNR is required over the length of a transient

fr = 10 # approx final rate  (after eventual downsampling )
decay_time = 1.  # length of typical transient in seconds 
use_cnn = False  # CNN classifier is designed for 2d (real) data
min_SNR = 3      # accept components with that peak-SNR or higher
rval_thr = 0.6   # accept components with space correlation threshold or higher
cnm2d.params.change_params(params_dict={'fr': fr,
                                      'decay_time': decay_time,
                                      'min_SNR': min_SNR,
                                      'rval_thr': rval_thr,
                                      'use_cnn': use_cnn})

cnm2d.estimates.evaluate_components(images, cnm2d.params, dview=dview)

In [ ]:
print(('Keeping ' + str(len(cnm2d.estimates.idx_components)) +
       ' and discarding  ' + str(len(cnm2d.estimates.idx_components_bad))))

In [ ]:
%%time
cnm2d.params.set('temporal', {'p': p})
cnm2d = cnm2d.refit(images)

In [ ]:
cnm2d.estimates.dims = dims2d



cnm2d.estimates.nb_view_components()

In [ ]:
# STOP CLUSTER
cm.stop_server(dview=dview)